In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import warnings
import contextlib
from datetime import datetime
import torch.multiprocessing as mp
import sys
mp.set_start_method('spawn')

In [2]:
from utils.datasets import *
from utils.eval import *
from utils.model import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_names = [
    "resnet50",
    "resnet18", "alexnet", "vgg16",
    "densenet", "inception", "googlenet", 
    "resnext50_32x4d", "wide_resnet50_2"
]

cuda


In [4]:
# freezed model
def get_original_model(net):
    if net == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        # models.resnet18(pretrained=True)
    elif net == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # model = models.resnet18(pretrained=True)
    elif net == "alexnet":
        # model = models.alexnet(pretrained=True)
        model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
    elif net == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        # model = models.vgg16(pretrained=True)
    elif net == "densenet":
        model = models.densenet161(weights=models.DenseNet161_Weights.DEFAULT)
        # model = models.densenet161(pretrained=True)
    elif net == "inception":
        model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        # model = models.inception_v3(pretrained=True)
    elif net == "googlenet":
        model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
        # model = models.googlenet(pretrained=True)
    elif net == "resnext50_32x4d":
        model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
        # model = models.resnext50_32x4d(pretrained=True)
    elif net == "wide_resnet50_2":
        model = models.wide_resnet50_2(weights=models.Wide_ResNet50_2_Weights.DEFAULT)
        # model = models.wide_resnet50_2(pretrained=True)
    else:
        raise ValueError("Invalid model name")
    for param in model.parameters():
        param.requires_grad = False
    # Remove the last fully connected layer
    if "inception" in net:
        model.aux_logits=False
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc = nn.Linear(2048, num_classes)
        model.fc.requires_grad = True
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
    if "alexnet" in net or "vgg" in net or  "mobilenet_v3_large" in net:
        last_fc_layer = model.classifier[-1]
        num_ftrs = last_fc_layer.in_features
        model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
        model.classifier[-1].requires_grad = True
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    elif "densenet" in net:
        last_fc_layer = model.classifier
        num_ftrs = last_fc_layer.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
        model.classifier.requires_grad = True
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc.requires_grad = True
        model.fc = nn.Linear(num_ftrs, num_classes)
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    return model,criterion, optimizer

In [11]:
num_epochs = 20
def train_net(net):
    model, criterion, optimizer = get_original_model(net)
    original_stderr = sys.stderr
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    tps, tns, fps, fns = [], [], [], []
    current_date = datetime.now().strftime('%Y-%m-%d')
    save_dir = os.path.join("saved_models/tl", current_date)
    os.makedirs(save_dir, exist_ok=True)
    
    sys.stderr = open(os.devnull, 'w')
    
    for timestamp in range(1, 290):
    # for timestamp in range(50, 51):
        debug = False
        if timestamp in  [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 289]:
            debug = True
        if debug:
            print("timestamp: ", timestamp)
        if "inception" in net:
            dataloaders = get_dataloaders(timestamp, inception=True, data_enrich=True)
        else:
            dataloaders = get_dataloaders(timestamp, data_enrich=True)
        model_trained, loss_values = train_model(model, criterion, optimizer, dataloaders, num_epochs=num_epochs, inception=("inception" in net), output=debug, debug=False)
        model_filename = os.path.join(save_dir, f"model_{net}_time_{timestamp}_epochs{num_epochs}.pt")
        torch.save(model_trained.state_dict(), model_filename)
    
        accuracy, precision, recall, f1, tp, tn, fp, fn = get_metrics(model_trained, dataloaders['test'], inception=("inception" in net))
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        if debug:
            print("Accuracy: ", accuracy)
        recall_list.append(recall)
        f1_list.append(f1)
        tps.append(tp)
        tns.append(tn)
        fps.append(fp)
        fns.append(fn)
    sys.stderr = original_stderr
    return accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns

In [ ]:
# for model in 
# accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns = train_net("googlenet")
model_names = [
    # "resnet50",
    # "resnet18", "alexnet", "vgg16",
    # "densenet", 
    # "inception", 
    # "googlenet", 
    "resnext50_32x4d", 
    "wide_resnet50_2"
]
acc_map = {}
pre_map = {}
rec_map = {}
f1_map = {}
tps_map = {}
tns_map = {}
fps_map = {}
fns_map = {}

for model_name in model_names:
    print(f"training {model_name}...")
    current_date = datetime.now().strftime('%Y-%m-%d')
    os.makedirs(f"./output/tl+rotation/{current_date}/{model_name}", exist_ok=True)
    show_ = False
    accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns = train_net(model_name)
    plot_metric(accuracy_list, 'Accuracy', model=model_name, save=True, show=show_, output=f"output/tl+rotation/{current_date}", num_epochs=num_epochs)
    plot_metric(precision_list, 'Precision', model=model_name, save=True, show=show_,output=f"output/tl+rotation/{current_date}", num_epochs=num_epochs)
    plot_metric(recall_list, 'Recall', model=model_name, save=True, show=show_,output=f"output/tl+rotation/{current_date}", num_epochs=num_epochs)
    plot_metric(f1_list, 'F1 Score', model=model_name, save=True, show=show_,output=f"output/tl+rotation/{current_date}", num_epochs=num_epochs)
    for time in [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 289]:
    # for time in [1]:
        plot_confusion_matrix(tps, tns, fps, fns, time, save=True, show=show_,output=f"./output/tl+rotation/{current_date}/{model_name}")
    acc_map[model_name] = accuracy_list
    pre_map[model_name] = precision_list
    rec_map[model_name] = recall_list
    f1_map[model_name] = f1_list
    tps_map[model_name] = tps
    tns_map[model_name] = tns
    fps_map[model_name] = fps
    fns_map[model_name] = fns

    print(f"{model_name} average accuracy:", sum(accuracy_list) / len(accuracy_list))

training resnet50...
timestamp:  1
Training complete in 0m 9s
Best val Acc: 0.806452
Accuracy:  0.75
timestamp:  20
Training complete in 0m 12s
Best val Acc: 0.741935
Accuracy:  0.75
timestamp:  40
Training complete in 0m 12s
Best val Acc: 0.870968
Accuracy:  0.96875
timestamp:  60
Training complete in 0m 12s
Best val Acc: 0.935484
Accuracy:  0.875
timestamp:  80
Training complete in 0m 11s
Best val Acc: 0.935484
Accuracy:  0.96875
timestamp:  100
Training complete in 0m 12s
Best val Acc: 0.967742
Accuracy:  0.9375
timestamp:  120
Training complete in 0m 12s
Best val Acc: 0.967742
Accuracy:  1.0
timestamp:  140
Training complete in 0m 11s
Best val Acc: 1.000000
Accuracy:  1.0
timestamp:  160
Training complete in 0m 10s
Best val Acc: 1.000000
Accuracy:  1.0
timestamp:  180
Training complete in 0m 10s
Best val Acc: 1.000000
Accuracy:  0.96875
timestamp:  200
Training complete in 0m 10s
Best val Acc: 1.000000
Accuracy:  1.0
timestamp:  220
Training complete in 0m 10s
Best val Acc: 1.00000

In [ ]:
avg_acc_map = {model: sum(acc) / len(acc) for model, acc in acc_map.items()}
avg_pre_map = {model: sum(pre) / len(pre) for model, pre in pre_map.items()}
avg_rec_map = {model: sum(rec) / len(rec) for model, rec in rec_map.items()}
avg_f1_map = {model: sum(f1) / len(f1) for model, f1 in f1_map.items()}
print("Average Accuracy:")
for model, avg_acc in avg_acc_map.items():
    print(f"{model}: {avg_acc}")

print("\nAverage Precision:")
for model, avg_pre in avg_pre_map.items():
    print(f"{model}: {avg_pre}")

print("\nAverage Recall:")
for model, avg_rec in avg_rec_map.items():
    print(f"{model}: {avg_rec}")

print("\nAverage F1 Score:")
for model, avg_f1 in avg_f1_map.items():
    print(f"{model}: {avg_f1}")
    
output_folder = f"./output/tl+rotation/{current_date}"
output_file = "average_metrics_2.txt"

# Combine average metric values for all models into a single string
combined_metrics = ""
for model in model_names:
    if model in avg_acc_map:
        combined_metrics += f"{model} - Average Accuracy: {avg_acc_map[model]}, Average Precision: {avg_pre_map[model]}, Average Recall: {avg_rec_map[model]}, Average F1 Score: {avg_f1_map[model]}\n"

# Write the combined metrics string to the output file
with open(os.path.join(output_folder, output_file), "w") as f:
    f.write(combined_metrics)
    for model in model_names:
        if model in avg_acc_map:
            f.write(f"\n{model}:\n")
            f.write("Accuracy: " + str(acc_map[model]) + "\n")
            f.write("Precision: " + str(pre_map[model]) + "\n")
            f.write("Recall: " + str(rec_map[model]) + "\n")
            f.write("F1 Score: " + str(f1_map[model]) + "\n")
            f.write("TPs: " + str(tps_map[model]) + "\n")
            f.write("TNs: " + str(tns_map[model]) + "\n")
            f.write("FPs: " + str(fps_map[model]) + "\n")
            f.write("FNs: " + str(fns_map[model]) + "\n")

In [ ]:
# plot_metric(accuracy_list, 'Accuracy', model="googlenet", save=True, output="output/tl")
# plot_metric(precision_list, 'Precision', model="googlenet", save=True, output="output/tl")
# plot_metric(recall_list, 'Recall', model="googlenet", save=True, output="output/tl")
# # plot_metric(f1_list, 'F1 Score', model="googlenet", save=True, output="output/tl")

In [ ]:
# for timestamp in range(1, 2):
# # for timestamp in range(1, 10):
#     print("timestamp: ", timestamp)
#     dataloaders = get_dataloaders(timestamp)
#     print(len(dataloaders['train'].dataset))
#     print(len(dataloaders['val'].dataset))
#     print(len(dataloaders['test'].dataset))